<a href="https://colab.research.google.com/github/laralex/Sk-DL2021-FinalProject/blob/main/notebooks/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Signal recovery from nonlinear distortion in optical communications
Deep Learning 2021 course at Skoltech, Russia.

A quickstart notebook to reproduce the experiments. Suggested to use on Google Colab.

<h3><span style="color:blue;">Configure only there</span></h3>

In [1]:
CONFIG_PATH = 'configs/mnist.yaml'

ON_GOOGLE_COLAB = False
MOUNT_GOOGLE_DRIVE = False # can't mount on Windows
GPUS_INDICES = None # None for CPU, list for multiple GPU

GIT_BRANCH = 'configs'
GIT_PULL = True

### Destination of the experimental results (logs, checkpoints)

In [2]:
import os
import sys

if MOUNT_GOOGLE_DRIVE:
    from google.colab import drive
    current_dir = '/content' if ON_GOOGLE_COLAB else '.'
    drive.mount(f'{current_dir}/drive')
    root_dir = f'{current_dir}/drive/MyDrive/DL2021_SignalRecoveryExperiments'
else:
    root_dir = '.'

%cd $root_dir 
git_root = !git rev-parse --show-toplevel
already_in_repo = os.path.exists(git_root[0])
if already_in_repo:
    root_dir = git_root[0]
    
root_dir

D:\masters\courses\4-deep-learning\project\repo\notebooks


'D:/masters/courses/4-deep-learning/project/repo'

### Our repository and prerequisites

In [3]:
if not already_in_repo:
    !git clone https://github.com/laralex/Sk-DL2021-FinalProject .
!git checkout $GIT_BRANCH

!pip install pytorch_lightning
!pip install jsonargparse

sys.path.append(root_dir)

M	README.md
M	configs/mnist.yaml
M	models/mnist_classifier.py
M	notebooks/training.ipynb


Already on 'configs'


### TensorBoard logs

In [4]:
%load_ext tensorboard
%tensorboard --logdir $root_dir/logs

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 12296.

### Launch training

In [5]:
# useful for contributors
if GIT_PULL:
    !git pull origin $GIT_BRANCH

Already up to date.


From https://github.com/laralex/Sk-DL2021-FinalProject
 * branch            configs    -> FETCH_HEAD


In [8]:
%reload_ext autoreload
%autoreload 1

%aimport train

In [ ]:
train.main(
    root_dir = root_dir,
    config_path = f"{root_dir}/{CONFIG_PATH}", 
    gpu_indices = GPUS_INDICES)